In [1]:
import os
from openai import OpenAI
import json


api_key = "sk-proj-"

client = OpenAI(api_key=api_key)

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

### Simple Chain

In [3]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI(api_key=api_key)
output_parser = StrOutputParser()

In [4]:
chain = prompt | model | output_parser

In [5]:
chain.invoke({"topic": "bears"})

"Why don't bears wear shoes? \n\nBecause they have bear feet!"

### More complex chains

In [16]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [11]:
from langchain_community.document_loaders import TextLoader

directory = "./texts/"
file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.txt')]


In [12]:
file_paths

['./texts/text2.txt', './texts/text1.txt']

In [13]:
loaders = [TextLoader(file_path) for file_path in file_paths]


In [14]:
all_documents = []
for loader in loaders:
    all_documents.extend(loader.load())

In [17]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(all_documents)
embeddings = OpenAIEmbeddings(api_key=api_key)
db = FAISS.from_documents(texts, embeddings)

In [18]:
retriever = db.as_retriever()

In [19]:
retriever.get_relevant_documents("Kim konser verecek?")

/home/dell/miniconda3/envs/aa/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='Iron Maiden\'in vokalisti Bruce Dickinson İstanbul\'da konser verecek\n\nron Maiden\'in vokalisti, yazar, müzisyen ve iş adamı Bruce Dickinson, 19 Temmuz\'da Küçükçiftlik Park\'ta müzikseverlerle buluşacak.\nDickinson\'a konserde, gitarda Roy Z, davulda Dave Moreno, basgitarda Tanya O\'Callaghan, klavyede ise Mistheria eşlik edecek.\nKonserin açılışını Cenk Durmazel\'in solistliğini üstlendiği Malt grubu yapacak.\nAlbümün ilk parçası "Afterglow of Ragnarok", 8 sayfalık bir çizgi romanla vinil şeklinde çıkmıştı. \nParça aynı zamanda, Iron Maiden\'ın "The Book Of Souls" albümünde yer alan "If Eternity Should Fail" parçasının Dickinson tarafından kaydedilen orijinal versiyonunu da içeriyor.\n\nÖmer Mirza Şeker ,Ali Özer\nKültür 08.07.2024', metadata={'source': './texts/text2.txt'}),
 Document(page_content='Milli kaleci Mert Günok\'tan EURO 2024 için veda mesajı\n\n2024 Avrupa Şampiyonası\'nda (EURO 2024) çeyrek finalde elenen A Milli Futbol Takımı\'nın kalecilerind

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [21]:
from langchain.schema.runnable import RunnableMap

In [22]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [23]:
chain.invoke({"question": "Kim konser verecek?"})

"Iron Maiden'in vokalisti Bruce Dickinson İstanbul'da konser verecek."

### Bind

In [36]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [37]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(api_key=api_key, temperature=0).bind(functions=functions)

In [38]:
runnable = prompt | model

In [27]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-74cebeed-a903-48fe-aa98-ad54afbb2aa7-0', usage_metadata={'input_tokens': 64, 'output_tokens': 16, 'total_tokens': 80})

### deneme

In [62]:
from langchain.schema.runnable import RunnableMap
from langchain.llms import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, ChatPromptOutputParser

# Şehir ismi ve posta kodu eşlemelerini içeren bir sözlük
city_to_postcode = {
    "Ankara": "06000",
    "Istanbul": "34000",
    "Izmir": "35000",
    # Daha fazla şehir ve posta kodu ekleyin
}

# Şehir ismini alarak posta kodunu döndüren fonksiyon
def get_postcode(city_name):
    return city_to_postcode.get(city_name, "Posta kodu bulunamadı")

# OpenAI API anahtarı ile model oluşturma
model = ChatOpenAI(api_key=api_key, temperature=0,functions=functions)
# Prompt şablonunu oluşturun
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Prompt ve Output Parser'ı oluşturun (bunlar sizin tanımladığınız olmalıdır)
prompt = ChatPromptTemplate(...)  # Prompt şablonunuza göre düzenleyin

# Mevcut retriever'ı kullanarak zincir oluşturun
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
    "postcode": lambda x: get_postcode(x["city"])  # Posta kodu fonksiyonunu doğrudan çağırın
}) | prompt | model | output_parser

# Soruyu ve şehir ismini içeren bir sorgu yapın
result = chain.invoke({"question": "Kim konser verecek?", "city": "Istanbul"})

# Sonucu yazdırın
print(result)


ImportError: cannot import name 'ChatOpenAI' from 'langchain.llms' (/home/dell/miniconda3/envs/aa/lib/python3.8/site-packages/langchain/llms/__init__.py)

In [61]:
# ChatOpenAI modelini fonksiyonlarla birlikte bağlayın
model = ChatOpenAI(api_key=api_key, temperature=0)

from langchain.schema.runnable import RunnableMap
# Prompt şablonunu oluşturun
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Output Parser oluşturun (kendi gereksinimlerinize göre düzenleyin)

# Mevcut retriever'ı kullanarak zincir oluşturun
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
    "postcode": lambda x: get_postcode(x["city"])  # Posta kodu fonksiyonunu doğrudan çağırın
}) | prompt | model | output_parser

# Soruyu ve şehir ismini içeren bir sorgu yapın
result = chain.invoke({"question": "Kim konser verecek?", "city": "Istanbul"})

# Sonucu yazdırın
print(result)

Iron Maiden'in vokalisti Bruce Dickinson İstanbul'da konser verecek.


In [56]:
response = model({
    "context": "Burada bağlam bilgisi yer alır",
    "question": "Kim konser verecek?"
})
print(response)


TypeError: 'RunnableBinding' object is not callable

In [47]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(api_key=api_key, temperature=0).bind(
    function_call={"name": "get_postcode"}, functions=[functions])

chain2 = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
}) | prompt | model | output_parser
result = chain2.invoke({"question": "Kim konser verecek?"})


BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'functions[0]': expected a function definition, but got an array instead.", 'type': 'invalid_request_error', 'param': 'functions[0]', 'code': 'invalid_type'}}

### function call and bind

In [64]:
from langchain_core.runnables import RunnablePassthrough


function = {
    "name": "solver",
    "description": "Formulates and solves an equation",
    "parameters": {
        "type": "object",
        "properties": {
            "equation": {
                "type": "string",
                "description": "The algebraic expression of the equation",
            },
            "solution": {
                "type": "string",
                "description": "The solution to the equation",
            },
        },
        "required": ["equation", "solution"],
    },
}
# Need gpt-4 to solve this one correctly
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Write out the following equation using algebraic symbols then solve it.",
        ),
        ("human", "{equation_statement}"),
    ]
)
model = ChatOpenAI(api_key=api_key,model='gpt-3.5-turbo', temperature=0).bind(
    function_call={"name": "solver"}, functions=[function]
)
runnable = {"equation_statement": RunnablePassthrough()} | prompt | model
runnable.invoke("x raised to the third plus seven equals 12")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"equation":"x^3 + 7 = 12","solution":"x = 1"}', 'name': 'solver'}}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 98, 'total_tokens': 119}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-629fb17c-575a-4b47-80fc-74a995e27bfd-0', usage_metadata={'input_tokens': 98, 'output_tokens': 21, 'total_tokens': 119})

In [66]:
response = runnable.invoke("x raised to the third plus seven equals 12")

In [73]:
response.additional_kwargs["function_call"]

{'arguments': '{"equation":"x^3 + 7 = 12","solution":"x = 1"}',
 'name': 'solver'}

#### Fallbacks avoid json error